In [1]:
#Imports for libraries used in video processing,pose estimations, and data handling 
import os
import cv2
import pandas as pd
import numpy as np
import mediapipe as mp

In [4]:
#Paths 
video_folder = "/Users/christinasupino/Desktop/DIVES/"
output_folder = "/Users/christinasupino/Desktop/DIVES/DIVE_CLIPS"
timestamp_path = "/Users/christinasupino/Desktop/DIVES/DIVE_TIMESTAMPS.csv"

os.makedirs(output_folder, exist_ok=True)

#Load timestamps 
df = pd.read_csv("/Users/christinasupino/Desktop/DIVES/DIVE_TIMESTAMPS.csv")

#Convert mm:ss to seconds
def time_to_seconds(t):
    t = t.replace("–", "-").strip()
    m, s = t.split(":")
    return int(m) * 60 + float(s)

#Process each row
for _, row in df.iterrows():
    dive_name=row["DiveName"]
    diver= row["Diver"]
    start_sec=time_to_seconds(row["Start"])
    end_sec=time_to_seconds(row["End"])

    video_path = os.path.join(video_folder, dive_name)
    if not os.path.exists(video_path):
        print(f" Missing video: {video_path}")
        continue

    #Open video
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)

    
    start_frame= int(start_sec * fps)
    end_frame =int(end_sec * fps)  # Fixed indentation here

    #Set starting frame
    cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)


    #Prepare output path
    out_path = os.path.join(output_folder, f"{diver}_{dive_name}")
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    width= int (cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height= int (cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    out = cv2.VideoWriter(out_path, fourcc, fps, (width, height))

    #Write frames in range
    frame_idx = start_frame
    while frame_idx < end_frame:
        ret, frame = cap.read()
        if not ret:
            break
        out.write(frame)
        frame_idx += 1

    cap.release()
    out.release()
    print(f"Saved: {out_path}")

print("All clips processed successfully!")

Saved: /Users/christinasupino/Desktop/DIVES/DIVE_CLIPS/FUNG_DIVE301B.mp4
Saved: /Users/christinasupino/Desktop/DIVES/DIVE_CLIPS/BROWN_DIVE301B.mp4
Saved: /Users/christinasupino/Desktop/DIVES/DIVE_CLIPS/MONROY_DIVE301B.mp4
Saved: /Users/christinasupino/Desktop/DIVES/DIVE_CLIPS/HUBERT_DIVE301B.mp4
Saved: /Users/christinasupino/Desktop/DIVES/DIVE_CLIPS/PALKHIVALA_DIVE301B.mp4
Saved: /Users/christinasupino/Desktop/DIVES/DIVE_CLIPS/FUNG_DIVE201B.mp4
Saved: /Users/christinasupino/Desktop/DIVES/DIVE_CLIPS/BROWN_DIVE201B.mp4
Saved: /Users/christinasupino/Desktop/DIVES/DIVE_CLIPS/MONROY_DIVE201B.mp4
Saved: /Users/christinasupino/Desktop/DIVES/DIVE_CLIPS/PALKHIVALA_DIVE201B.mp4
Saved: /Users/christinasupino/Desktop/DIVES/DIVE_CLIPS/FUNG_DIVE103B.mp4
Saved: /Users/christinasupino/Desktop/DIVES/DIVE_CLIPS/BROWN_DIVE103B.mp4
Saved: /Users/christinasupino/Desktop/DIVES/DIVE_CLIPS/MONROY_DIVE103B.mp4
Saved: /Users/christinasupino/Desktop/DIVES/DIVE_CLIPS/HUBERT_DIVE103B.mp4
Saved: /Users/christinasup

In [5]:
#Run pose estimation on each clip and export all 33 landmark coordinates per frame

#MediaPipe setup
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(
    static_image_mode=False,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)


#Folder with dive clips
clips_folder = "/Users/christinasupino/Desktop/DIVES/DIVE_CLIPS"
video_files = [f for f in os.listdir(clips_folder) if f.endswith(".mp4")]

#Store all keypoints data
all_keypoints = []

for video_file in video_files:
    video_path = os.path.join(clips_folder, video_file)
    cap = cv2.VideoCapture(video_path)
    
    if not cap.isOpened():
        print(f" Could not open: {video_file}")
        continue
    
    print(f"Processing {video_file}...")
    frame_idx = 0
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        # Perform pose estimation
        results = pose.process(image_rgb)
        
        if results.pose_landmarks:
            row_data = [video_file, frame_idx]
            for lm in results.pose_landmarks.landmark:
                row_data.extend([lm.x, lm.y, lm.z, lm.visibility])
            all_keypoints.append(row_data)
        
        frame_idx += 1
    
    cap.release()

pose.close()

#Save to CSV
columns = ["Video", "Frame"]
for i in range(33):
    columns.extend([f"{i}_x", f"{i}_y", f"{i}_z", f"{i}_v"])


keypoints_df = pd.DataFrame(all_keypoints, columns=columns)
keypoints_df.to_csv("/Users/christinasupino/Desktop/DIVES/DIVE_KEYPOINTS.csv", index=False)


print("Keypoints extraction complete and saved to CSV!")

    

I0000 00:00:1764211851.560808 7294995 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 90.5), renderer: Apple M1
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1764211851.671211 7299103 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1764211851.692628 7299106 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1764211851.720562 7299105 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.


Processing PALKHIVALA_DIVE105B.mp4...
Processing FUNG_DIVE105B.mp4...
Processing MONROY_DIVE201B.mp4...
Processing MONROY_DIVE105B.mp4...
Processing PALKHIVALA_DIVE201B.mp4...
Processing HUBERT_DIVE301B.mp4...
Processing FUNG_DIVE201B.mp4...
Processing HUBERT_DIVE403B.mp4...
Processing PALKHIVALA_DIVE103B.mp4...
Processing FUNG_DIVE103B.mp4...
Processing BROWN_DIVE403B.mp4...
Processing BROWN_DIVE301B.mp4...
Processing MONROY_DIVE103B.mp4...
Processing MONROY_DIVE403B.mp4...
Processing MONROY_DIVE301B.mp4...
Processing BROWN_DIVE103B.mp4...
Processing HUBERT_DIVE105B.mp4...
Processing FUNG_DIVE301B.mp4...
Processing PALKHIVALA_DIVE403B.mp4...
Processing FUNG_DIVE403B.mp4...
Processing PALKHIVALA_DIVE301B.mp4...
Processing BROWN_DIVE105B.mp4...
Processing HUBERT_DIVE103B.mp4...
Processing BROWN_DIVE201B.mp4...
Keypoints extraction complete and saved to CSV!


In [10]:
#Estimate dive phases using changes in average Z landmark coordinates
#Compute dynamic dive phases (start, take-off, flight, entry) per video
def compute_dive_phases(df):
    keypoint_z_cols = [col for col in df.columns if "_z" in col]
    dive_phase_indices = []

    #Loop for each video
    for video, video_df in df.groupby("Video"):
        
        z_values = video_df[keypoint_z_cols].mean(axis=1).values
        dz = np.diff(z_values)

        #Start:first frame
        start_idx = 0
        #Take-off:Frame with maximum upward velocity
        takeoff_idx = int(np.argmax(dz))
        #Entry:Frame with maximum downward velocity
        entry_idx = int(np.argmin(dz))
        #Last frame
        last_idx = len(video_df) - 1

        # Define flight phase
        flight_start_idx = takeoff_idx + 1
        flight_end_idx = max(takeoff_idx + 1, entry_idx - 1)

        dive_phase_indices.append({
            "Video": video,
            "StartFrame": start_idx,
            "TakeoffFrame": takeoff_idx,
            "FlightStartFrame": flight_start_idx,
            "FlightEndFrame": flight_end_idx,
            "EntryFrame": entry_idx,
            "LastFrame": last_idx
        })

    return pd.DataFrame(dive_phase_info)


#Compute and save phase info  keypoints dataframe
df_dive_phases = compute_dive_phases(keypoints_df) 
df_dive_phases.to_csv("DIVE_PHASES.csv", index=False)

print("Dive phases computed and saved to DIVE_PHASES.csv")

Dive phases computed and saved to DIVE_PHASES.csv
